In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from utils import draw_tracking_result, save_tracking_results
from tracks import load_detections, track_iou, create_similarity_matrix, associate_detections_to_tracks, update_tracks, initialize_new_track, look_forward
from KalmanFilter import KalmanFilter

###  Loading det object instances

In [2]:
det_file_path = '../data/det/det.txt'
detections_df = load_detections(det_file_path)
detections_df.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1


#### Interface (main loop)

In [3]:
# Main loop
image_frames_path = '../data/img1/' 

# Define properties for the output video
fps = 40
frame_size = (1920, 1080)  # Adjust this to match the size of your frames
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, frame_size)
delay = int(1000/fps)

# Initialize tracking variables
tracks_history = {}
frame_number = 1
sigma_iou = 0.5
CONF_THRESH = 20.0
nb_step = 2

tracking_file = 'track.txt'
if os.path.isfile(tracking_file):
    open(tracking_file, 'w').close()
else:
    open(tracking_file, 'x')

# Process each image frame
for filename in sorted(os.listdir(image_frames_path)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(image_frames_path, filename)
        frame = cv2.imread(frame_path)

        # Looking forward
        tracks_history = look_forward(
            detections_df,
            tracks_history,
            frame_number,
            nb_step=5,
            sigma_iou=sigma_iou,
            conf_threshold=CONF_THRESH
        )
        
        # Draw tracking
        frame_with_tracking = draw_tracking_result(frame, tracks_history[frame_number])
        # Save the tracking results
        save_tracking_results(tracks_history[frame_number], tracking_file, frame_number)

        # Show the frame
        cv2.imshow('Tracking', frame_with_tracking)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
        out.write(frame_with_tracking)

        frame_number += 1

out.release()
cv2.destroyAllWindows()

frame 2: (2, 4), current_boxes x tracks
det_indices: 2, track_indices: 2
frame 3: (3, 8), current_boxes x tracks
det_indices: 3, track_indices: 3
frame 4: (4, 24), current_boxes x tracks
det_indices: 4, track_indices: 4
frame 5: (3, 64), current_boxes x tracks
det_indices: 3, track_indices: 3
frame 6: (6, 48), current_boxes x tracks
det_indices: 6, track_indices: 6
matches: 3, unmatched_detections: 3, unmatched_tracks: 45
frame 7: (3, 96), current_boxes x tracks
det_indices: 3, track_indices: 3
matches: 3, unmatched_detections: 0, unmatched_tracks: 93
frame 8: (3, 48), current_boxes x tracks
det_indices: 3, track_indices: 3
matches: 3, unmatched_detections: 0, unmatched_tracks: 45
frame 9: (4, 48), current_boxes x tracks
det_indices: 4, track_indices: 4
matches: 3, unmatched_detections: 1, unmatched_tracks: 45
frame 10: (2, 64), current_boxes x tracks
det_indices: 2, track_indices: 2
matches: 2, unmatched_detections: 0, unmatched_tracks: 62
frame 11: (2, 32), current_boxes x tracks
det